In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load Data
# (ตรวจสอบชื่อไฟล์ให้ตรงกับที่มีในเครื่องนะครับ)
df = pd.read_csv('train.csv') 

print(f"Original Data Shape: {df.shape}")
display(df.head(3))

Original Data Shape: (9800, 18)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62


In [ ]:
# 2. Convert Date Columns
# แปลงวันที่ให้เป็น Format ที่ถูกต้อง (ระบุ dayfirst=True เพราะในไฟล์เป็น DD/MM/YYYY)
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, format='%d/%m/%Y', errors='coerce')
df['Ship Date'] = pd.to_datetime(df['Ship Date'], dayfirst=True, format='%d/%m/%Y', errors='coerce')

# ลบแถวที่วันที่ผิดปกติ (NaT) ที่แปลงค่าไม่ได้
df.dropna(subset=['Order Date', 'Ship Date'], inplace=True)

# 3. Handle Missing Values & Duplicates
# ลบแถวที่มีค่าว่าง
df.dropna(inplace=True)

# ลบข้อมูลซ้ำ
if df.duplicated().sum() > 0:
    df.drop_duplicates(inplace=True)

# 4. Logic Validation
# เช็คความถูกต้อง: วันส่งของ (Ship Date) ต้องไม่เกิดก่อนวันสั่ง (Order Date)
df = df[df['Ship Date'] >= df['Order Date']]

print(f"Cleaned Data Shape: {df.shape}")

In [ ]:
# 5. Feature Engineering
df['Order Year'] = df['Order Date'].dt.year
df['Order Month'] = df['Order Date'].dt.month_name()

# คำนวณระยะเวลาส่งของ (Delivery Days)
df['Delivery Days'] = (df['Ship Date'] - df['Order Date']).dt.days

# 6. Visualization
df.to_csv('cleaned_superstore.csv', index=False)
print("✅ Saved 'cleaned_superstore.csv'")
display(df[['Order ID', 'Delivery Days']].head())

In [ ]:
# 7. R-F-M Analysis
# 1. กำหนดวันล่าสุดในข้อมูล (สมมติว่าเป็นวัน  ถัดไปจากวันสั่งซื้อสูงสุด)
latest_date = df['Order Date'].max() + dt.timedelta(days=1)

# 2. คำนวณค่า R-F-M รายลูกค้า
rfm = df.groupby('Customer ID').agg({
    'Order Date': lambda x: (latest_date - x.max()).days, # Recency: ห่างจากวันล่าสุดกี่วัน
    'Order ID': 'count',                                  # Frequency: จำนวนครั้งที่ซื้อ
    'Sales': 'sum'                                        # Monetary: ยอดเงินรวม
}).reset_index()

# เปลี่ยนชื่อคอลัมน์ให้เข้าใจง่าย
rfm.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary']

print("RFM Table Calculated:")
display(rfm.head())

In [ ]:
# 3. การให้คะแนน R-F-M (1-5)
labels = range(1, 6)
# Recency: ค่าน้อย = ดี
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=5, labels=list(range(5, 0, -1))) 
# Frequency: ค่ามาก = ดี
rfm['F_Score'] = pd.qcut(rfm['Frequency'], q=5, labels=labels, duplicates='drop')
# Monetary: ค่ามาก = ดี
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=5, labels=labels)

# รวมคะแนน R-F-M
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].sum(axis=1)

# 4. การแบ่งกลุ่มลูกค้า (Segmentation)
def segment_customer(score):
    if score >= 13:
        return 'VIP'             # ลูกค้าชั้นดี (คะแนน 13-15)
    elif score >= 9:
        return 'Potential'       # ลูกค้าศักยภาพ (คะแนน 9-12)
    elif score >= 5:
        return 'Needs Attention' # ลูกค้าทั่วไป/ต้องดูแล (คะแนน 5-8)
    else:
        return 'Lost'            # ลูกค้าที่หายไปแล้ว (คะแนนต่ำกว่า 5)


rfm['Segment'] = rfm['RFM_Score'].apply(segment_customer)

# บันทึกไฟล์ RFM 
rfm.to_csv('rfm_analysis.csv', index=False)
print(" Saved 'rfm_analysis.csv'")
print(rfm['Segment'].value_counts())